<a href="https://colab.research.google.com/github/Giffy/MongoDB_PyMongo_Tutorial/blob/master/1_1_First_steps_with_MongoDB_PyMongo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First steps with MongoDB and PyMongo


*   Connection to Mongo
*   Basic database, collection management
*   CRUD  one document
*   CRUD  many documents





# 1. Install PyMongo (if required)

In [0]:
# Instaling PyMongo, this is the interface to connect to MongoDB with Python
! python -m pip install pymongo==3.7.2

#2. Import PyMongo and establish connection

In [0]:
import datetime                            # Imports datetime library

import pymongo
from pymongo import MongoClient
# uri (uniform resource identifier) defines the connection parameters 
# 
uri = 'localhost:27017'
# start client to connect to MongoDB server 
client = MongoClient( uri )

In [0]:
client.stats                                # .stats  show details about the client

# 3. Basic Database operations

In [0]:
# Show existing database names
client.list_database_names()

In [0]:
# Set database name to work with. If it doesn't exist, it will be created as soon as one document is added.
db = client.people

In [0]:
# Delete a database
client.drop_database('people')

#4. Basic Collection operations

In [0]:
# Create a new collection. We can create the collection or leave to MongoDB to create it as soon as a document is generated.
db.create_collection('adressbook')       # Optional collection creation 

In [0]:
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
list (db.list_collections())
# empty list '[]' means that there are not collections in database

In [0]:
# Set the collection to work with
collection = db.adressbook
collection.insert_one({'name' : 'jordi'})     # Insert one item to create the collection
list (db.list_collections())                  # Show the existing collections

In [0]:
# Rename a collections
db.adressbook.rename('addressbook')
collection = db.addressbook                   # Set the collection to work with
list (db.list_collections())                  # Show the existing collections

In [0]:
# Delete collection
db.drop_collection('addressbook')

# 5. Basic Collection operations

##5.1 Query Operators - Create Read Update Delete  (one document)

### 5.1.1 Create a document with:  insert_one()

To insert a document can be done using: insert_one and JavaScript notation  { 'attributeName1'  :  'content1', 'attributeName2'  :  'content2', ... }


In [0]:
#@title Contact

Name = "Jordi Perez" #@param {type:"string"}
Age = 34 #@param {type:"slider", min:10, max:80, step:1}
Gender = "Male" #@param ["Male", "Female"]
Likes_Python = "Yes" #@param ["Yes", "No"]
if Likes_Python is "Yes":
  Likes_Python = True
else:
  Likes_Python = False


#@markdown Address
Street = "Torrent de l'Olla" #@param {type:"string"}
Number = 70 #@param {type:"integer"}
City = "Barcelona" #@param {type:"string"}
PostalCode = "08012" #@param {type:"string"}

In [0]:
data = {  'name' : Name ,                                    # String 
          'age' : Age,                                       # Integer
          'gender' : Gender,                                 # String 
          'likes_python' : Likes_Python,                     # Boolean
          'address': {
              'street' : Street,                             # String ( special character with escape \ )
              'number' : Number,                             # Integer
              'city' : City,                                 # String 
              'floor' : None,                                # Null 
              'postalcode' : PostalCode,                     # String containing a number
          'favouriteFruits': ['banana','pineapple','orange']         # Array  

          }        
       }

insert_result = collection.insert_one( data)

In [0]:
insert_result.acknowledged   # Confirms that insert is successful

In [0]:
insert_result.inserted_id     # Shows the document ID 

###5.1.2 Read document with:  find()

To read a document can be done using: find and JavaScript notation  {  'attribute1 containing the key'  :  'key1 to find' }


In [0]:
list ( collection.find() )                 # gets all data of collection

In [0]:
list( collection.find( {'_id' : insert_result.inserted_id } ))  # Find the inserted document using the objectID

In [0]:
list ( collection.find( {'name' : Name } ))        # find, can use one key or more 

In [0]:
list ( collection.find( {'address.city' : City } ))        # find, can use one key or more 

In [0]:
list ( collection.find().limit(1) )        # gets a Limited set of documents

In [0]:
list ( collection.find().skip(1) )        # gets all documents skipping first

###5.1.3 Update a document with:  update_one()

In [0]:
## Update an existing document
update_result = collection.update_one( {'name' : Name}, {'$set' : { 'age' : 30 }} ) 
list (collection.find( {'name' : Name } ))

In [0]:
update_result.raw_result

In [0]:
## Insert a new document with update, will avoid to crash during insert if document already exist
insert_result = collection.update_one( {'name' : 'Javi Gonzalez'}, {'$set' : { 'age' : 30 }}, upsert= True )
list (collection.find( {'name' : 'Javi Gonzalez'} ))

In [0]:
update_result.acknowledged

###5.1.4 Delete a document with:  delete_one()

In [0]:
delete = collection.delete_one({'name': 'Javi Gonzalez'})

In [0]:
delete.deleted_count   # informs that 1 document has been deleted

In [0]:
collection.delete_one({'name': Name})

##5.2 Create Read Update Delete  (many document)

###5.2.1 Create a document with:  insert_many()

To insert many documents can be done using: insert_many and JavaScript notation  { 'attributeDocument1'  :  'content1' } , { 'attributeDocument2'  :  'content2'},{ ... }

In [0]:
import datetime
collection.insert_many(  [                          # <---- start a list with [
##  Insert Document 1
  {
  'name': 'Jordi Gonzalez',
  'age': 25,
  'likes_python': True,
  'registered': datetime.datetime(2015, 2, 11, 4, 22, 39),
  'address': {
      'street': 'Torrent de l\'Olla',
      'number': 70,
      'floor': None,
      'city': 'Barcelona',
      'postalCode': '08012'
             },
  'height':  1.72,
  'favouriteFruits': ['banana','pineapple','orange']
  },

##  Insert Document 2
  {
  'name': 'Maria Smith',
  'age': 30,
  'likes_python': True,
  'registered': datetime.datetime(2016, 4, 23, 7, 34, 12),
  'address': {
      'street': 'Numancia',
                                                     ##  missing number
                                                     ##  missing floor
      'city': 'Barcelona',
      'postalCode': '08029'
             },
  'height':  1.56,
  'favouriteFruits': ['lemon','pineapple']
  }
  ]   )                                              # <---- finalize the list ] 

###5.2.2 Read many documents with:  find()

To read a document can be done using: find and JavaScript notation  {  'attribute1 containing the key'  :  'key1 to find' }


In [0]:
list ( collection.find( {'$or': [ {'name': 'Jordi Gonzalez'},{'name': 'Maria Smith'} ]}))        # find 

###5.2.3 Update many documents with:  update_many()

In [0]:
collection.update_many( {'isActive': True }, {'$set' : { 'isActive': False }} )
list (collection.find( ))                       # List all documents

###5.2.4 Delete many documents with:  delete_many()

In [0]:
delete = collection.delete_many({'likes_python': True})    # deletes as many documents as the filter
list (collection.find( ))                                  # List all documents

In [0]:
delete.deleted_count   # items deleted

## Congratulations,  you finished the part 1 !!
Find additional notebooks to learn pymongo in http://www.github.com/giffy